In [7]:
import os
import shutil
import tempfile
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# CONFIG
base_dir = "uncentred_ternary_100_stratified4fold_1000per_seed3888 copy"
quadrants = ["Q1", "Q2", "Q3", "Q4"]
img_size = (224, 224)
batch_size = 32
epochs = 30

# Image Generators
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=10, zoom_range=0.1)
test_datagen = ImageDataGenerator(rescale=1./255)

# Merge training folders for each fold
def merge_training_dirs(train_dirs):
    temp_dir = tempfile.mkdtemp()
    class_names = ['Empty', 'Non-Tumor', 'Tumor']
    for class_name in class_names:
        os.makedirs(os.path.join(temp_dir, class_name), exist_ok=True)
    for train_dir in train_dirs:
        for class_name in class_names:
            src_dir = os.path.join(train_dir, class_name)
            dst_dir = os.path.join(temp_dir, class_name)
            for file in os.listdir(src_dir):
                shutil.copy(os.path.join(src_dir, file), dst_dir)
    return temp_dir

# Build the CNN model using VGG19
def build_model():
    base = VGG19(include_top=False, input_shape=(*img_size, 3), weights='imagenet')
    base.trainable = False  # Freeze base layers

    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(3, activation='softmax')(x)

    model = Model(inputs=base.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# For collecting fold metrics
all_histories = []
all_reports = []
best_val_loss = float('inf')  # Initialize with an infinitely large number
best_model = None  # To keep track of the best model

for i, test_fold in enumerate(quadrants):
    print(f"\n=== Fold {i+1}: Testing on {test_fold} ===")
    train_folds = [q for q in quadrants if q != test_fold]
    train_dirs = [os.path.join(base_dir, fold) for fold in train_folds]
    valid_dir = os.path.join(base_dir, test_fold)

    merged_train_dir = merge_training_dirs(train_dirs)

    train_generator = train_datagen.flow_from_directory(
        merged_train_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical'
    )

    val_generator = test_datagen.flow_from_directory(
        valid_dir, target_size=img_size, batch_size=batch_size,
        class_mode='categorical', shuffle=False
    )

    model = build_model()

    callbacks = [
        EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss'),
        ModelCheckpoint(f'best_model_fold_{i+1}.h5', save_best_only=True, monitor='val_loss')
    ]

    history = model.fit(train_generator, epochs=epochs, validation_data=val_generator, callbacks=callbacks)
    all_histories.append(history.history)

    # Predict and evaluate
    val_preds = model.predict(val_generator)
    y_pred = np.argmax(val_preds, axis=1)
    y_true = val_generator.classes
    class_labels = list(val_generator.class_indices.keys())

    print(f"Classification Report for Fold {i+1}:")
    report = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
    all_reports.append(report)
    print(classification_report(y_true, y_pred, target_names=class_labels))

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels, cmap='Blues')
    plt.title(f'Confusion Matrix - Fold {i+1}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Save the best model after each fold (check for better validation loss)
    fold_val_loss = min(history.history['val_loss'])
    if fold_val_loss < best_val_loss:
        best_val_loss = fold_val_loss
        best_model = model

    # Plot Loss/Accuracy
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'Loss - Fold {i+1}')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f'Accuracy - Fold {i+1}')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Save the best model after cross-validation
if best_model:
    best_model.save('best_model_VGG19.h5')
    print("\nBest model saved as 'best_model_VGG19.h5'")

# Average metrics over folds
avg_accuracy = np.mean([max(hist['val_accuracy']) for hist in all_histories])
print(f"\n=== Final Cross-Validation Accuracy: {avg_accuracy:.4f} ===")

# (Optional) Average classification report
import pandas as pd
df_reports = pd.DataFrame([r['accuracy'] for r in all_reports], columns=['accuracy'])
print("\nAverage Classification Accuracy per Fold:")
print(df_reports)



=== Fold 1: Testing on Q1 ===
Found 3384 images belonging to 3 classes.
Found 1128 images belonging to 3 classes.
Epoch 1/30
 30/106 [=======>......................] - ETA: 12:14 - loss: 0.9814 - accuracy: 0.4896